In [21]:
# First create the kh2lib object
from kh2lib.kh2lib import kh2lib
import os, json
lib = kh2lib()
if not os.path.isdir("workspace"):
    os.mkdir("workspace")

# What I write -> what you get in game
# UNCOMMENT DUMMY
# ITEMMAP = {ITEMMAP[i]:i for i in ITEMMAP}
    
ITEMS = ["Scan", "Guard", "Dodge Roll LV1", "Dodge Roll LV2", "Dodge Roll LV3"] + ["Reflect Element" for _ in range(3)] + ["Decisive Pumpkin"] + ["Negative Combo", "Negative Combo"] + \
    ["Limit Form"] + ["Feather Charm (Summon Peter Pan)"]

lib.give_to_sora(hp=120, mp=100, items=ITEMS)
# Because the localset seems to control where you go, I wonder if area/entrance control your last entered area???????

# 73 - Final Xemnas
# 58 - luxord
# 56 - saix
# 67 terra
lib.spawn_in("EH", "15", localset="58", entrance="0", jumptype="2", fadetype="1", setflags=[ "0x84A", ])

['unpack', '-o', 'C:\\Users\\12sam\\Documents\\GitHub\\kh2lib\\examples\\Using kh2lib\\plrptesting\\tempbattle', 'C:\\Users\\12sam\\Desktop\\KH_Games\\KH2\\ISO\\export\\KH2\\00battle.bin']
(None, None)
['pack', '-o', 'C:\\Users\\12sam\\Desktop\\KH_Games\\KH2\\ISO\\export\\KH2\\00battle.bin', 'C:\\Users\\12sam\\Documents\\GitHub\\kh2lib\\examples\\Using kh2lib\\plrptesting\\tempbattle\\00battle.bin.json']
(None, None)
['unpack', '-o', 'C:\\Users\\12sam\\Documents\\GitHub\\kh2lib\\examples\\Using kh2lib\\plrptesting\\tempard', 'C:\\Users\\12sam\\Desktop\\KH_Games\\KH2\\ISO\\export\\KH2\\ard\\tt01.ard']
(None, None)
['compile', '-o', 'C:\\Users\\12sam\\Documents\\GitHub\\kh2lib\\examples\\Using kh2lib\\plrptesting\\tempard\\evt.script', 'C:\\Users\\12sam\\Documents\\GitHub\\kh2lib\\examples\\Using kh2lib\\plrptesting\\tempard\\spawnscript']
(None, None)
['pack', '-o', 'C:\\Users\\12sam\\Desktop\\KH_Games\\KH2\\ISO\\export\\KH2\\ard\\tt01.ard', 'C:\\Users\\12sam\\Documents\\GitHub\\kh2lib\

In [ ]:
lib.reset_git()

In [ ]:
lib.patch_game(fromgit=True, useModManager=False)

In [ ]:
v = {}
for va in inventory.values():
    if va not in v:
        v[va] = 0
    v[va] += 1
for k in v:
    if v[k] > 1:
        print(k)

In [ ]:
inventory = {}
for line in open("inventory"):
    parts = [i.strip() for i in line.split("|")]
    inventory[int(parts[1])] = parts[2]
characters = {}
for line in open("characters"):
    parts = [i.strip() for i in line.split("|")]
    characters[int(parts[1])] = parts[2]
def lookupChar(idt):
    if idt in characters:
        return characters[idt]
    return idt
def lookupCharId(name):
    if type(name) == type(15):
        return name
    for k in characters:
        if characters[k] == name:
            return k
def lookupInv(idt_bytearr):
    idt_arr = list(idt_bytearr)
    idt_arr_bak = str(idt_arr)
    name = ""
    if idt_arr[0] >= 128:
        idt_arr[0] -= 128
        name = "EQ"
    idt = idt_arr[1] + 255*idt_arr[0]
    if idt in inventory:
        return name + inventory[idt]
    return idt_arr
def lookupInvNum(item):
    if type(item) == list:
        return item[::-1]
    itembytes = [0,0]
    if item.startswith("EQ"):
        itembytes[0] = 128
        item = item[2:]
    idt = None
    for k in inventory:
        if inventory[k] == item:
            idt = k
    if not idt:
        raise Exception("Item not found: {}".format(item))
    itembytes[0] = itembytes[0] + idt // 255 # sig byte
    itembytes[1] = idt % 255 # insig byte
    return itembytes[::-1]

In [ ]:
lookupInvNum("EQAuto Limit (I)")

In [ ]:
568 // 255

In [ ]:
255+162

In [ ]:
lib.openkh.bar_extract(os.path.join(lib.gamedir, "KH2", "00battle.bin"), os.path.join(os.getcwd(), "workspace", "battle"))

In [ ]:
import os
plrppth = os.path.join("workspace", "battle", "plrp.list")
data = bytearray(open(plrppth, "rb").read())
toint = lambda k: int(''.join([hex(i)[2:].zfill(2) for i in k]),16)
HEADER_LEN = 0x8

In [ ]:
subdata = data[8:]
character_records = []
for i in range(0, len(subdata), 128):
    d = {
        "unkid": toint(subdata[i:i+2]),
        "character": lookupChar(toint(subdata[i+2:i+3])),
        "hp": toint(subdata[i+3:i+4]),
        "mp": toint(subdata[i+4:i+5]),
        "ap": toint(subdata[i+5:i+6]),
        "armorslots": toint(subdata[i+6:i+7]),
#         "armorslots2": toint(subdata[i+7:i+8]),
        "accessoryslots": toint(subdata[i+8:i+9]),
#         "accessoryslots2": toint(subdata[i+9:i+10]),
        "itemslots": toint(subdata[i+10:i+11]),
#         "itemslots2": toint(subdata[i+11:i+12]),
#         "items": [lookupInv(toint(subdata[n:n+1])) for n in range(i+9, i+128, 1) if not toint(subdata[n:n+2]) == 0 ]
        "items": [lookupInv(subdata[n:n+2][::-1]) for n in range(i+12, i+128, 2) if not toint(subdata[n:n+2]) == 0]
#         "items": [toint(subdata[n:n+2][::-1]) for n in range(i+12, i+128, 2)]
    }
    character_records.append(d)
import yaml
yaml.dump(character_records, open("character_records.yaml", "w"))

In [ ]:
[i for i in data[:HEADER_LEN]]

In [ ]:
import yaml
cr = data[:HEADER_LEN]
for c in yaml.load(open("character_records_g.yaml")):
    unkid = hex(c["unkid"])[2:].zfill(4)
    cr.append(int(unkid[:2], 16))
    cr.append(int(unkid[2:], 16))
    character = hex(lookupCharId(c["character"]))[2:].zfill(2)
    cr.append(int(character, 16))
    cr.append( c["hp"])
    cr.append( c["mp"])
    cr.append( c["ap"])
    cr.append( c["armorslots"])
    cr.append( c["armorslots"])
    cr.append( c["accessoryslots"])
    cr.append( c["accessoryslots"])
    cr.append( c["itemslots"])
    cr.append( c["itemslots"])
    if len(c["items"]) > 58:
        raise Exception("Too many items")
    items = c["items"] + [[0,0] for _ in range(58-len(c["items"]))]
    for item in items:
        for subbyte in lookupInvNum(item):
            cr.append( subbyte )
plrppthnew = os.path.join("workspace", "battle", "plrp.list")
open(plrppthnew, "wb").write(cr)
lib.openkh.bar_build(os.path.join(os.getcwd(), "workspace", "battle", "00battle.bin.json"), os.path.join(lib.gamedir, "KH2", "00battle.bin"))

In [ ]:
lib.reset_git()